https://www.youtube.com/watch?v=n_Pg-wSRGyI

In [ ]:
!pip install visdom

     |████████████████████████████████| 686kB 7.8MB/s 
     |████████████████████████████████| 204kB 23.0MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655250 sha256=b6c3c43cd6bca17ed8b72ac25bbaf90101b523916076b22281283b329e7e8f4c
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=10af7a4a439b7f12867cc9d28a4c62e4a8b9f473bb48957c76ebf4297c9326f3
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torch.autograd import Variable
import visdom
import torch.optim as optim
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data',
                            train=True,
                            download=True,
                            transform=transform)
testset = datasets.CIFAR10(root='./data',
                           train=False,
                           download=True,
                           transform=transform)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 5),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3),
            nn.Conv2d(128, 128, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3),
            nn.MaxPool2d(2)
        )
        self.layer2_1 = nn.Sequential(
            nn.Conv2d(256, 512, 7, 1, 2),   # nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        self.layer2_2 = nn.Sequential(
            nn.Conv2d(256, 512, 5, 1, 1),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        self.layer2_3 = nn.Sequential(
            nn.Conv2d(256, 512, 3),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(3*64*4*4, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 10)
        )

    def forward(self, x):
        print(x.data.shape)
        x = self.layer1(x)
        x1 = self.layer2_1(x)
        x2 = self.layer2_2(x)
        x3 = self.layer2_3(x)
        x = torch.cat((x1,x2,x3), dim=1)    # BxCxHxW --> dim=1 --> C를 기준으로 concat
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
a = torch.rand(1, 3, 32, 32)
print(a.shape)

torch.Size([1, 3, 32, 32])


In [ ]:
network = Net()
out = network(a)
print(out.shape)

torch.Size([1, 3, 32, 32])
torch.Size([1, 10])


In [ ]:
print(out)

tensor([[-0.0092, -0.0131, -0.0228,  0.0121,  0.0174, -0.0039,  0.0230,  0.0303,
         -0.0070,  0.0248]], grad_fn=<AddmmBackward>)


In [ ]:
torch.save(network.state_dict(), './cnn.pth')

In [ ]:
model = Net()

In [ ]:
model.load_state_dict(torch.load('./cnn.pth'))

<All keys matched successfully>

In [ ]:
out = model(a)
print(out)

torch.Size([1, 3, 32, 32])
tensor([[-0.0092, -0.0131, -0.0228,  0.0121,  0.0174, -0.0039,  0.0230,  0.0303,
         -0.0070,  0.0248]], grad_fn=<AddmmBackward>)
